# Import dependencies

In [1]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans, AffinityPropagation
import warnings
warnings.filterwarnings("ignore")
import numpy as np

import torch
from torch.utils.tensorboard import SummaryWriter

from clustering import clustering_classification, test_classifier, write, save_metrics_to_dict, encode_categorical_features, import_dataset

# For logging 

In [2]:
writer = SummaryWriter()

train_acc_dict = {}
train_f1_dict = {}
test_acc_dict = {}
test_f1_dict = {}

In [3]:
# Launch tensorboard
# images=21 change this to how many datasets you use
%tensorboard --logdir runs/ --port 6006 --samples_per_plugin images=21
# If in use (Mac) use to find the process PID
% lsof -i :6006
# Kill the process with 
% kill -9 <PID>
# Then launch using bash with first command

UsageError: Line magic function `%tensorboard` not found.


# Config Params

In [4]:
TEST_SIZE = 0.2
RANDOM_SEED = 42
K_FOLDS = 5

# Our chosen datasets
TODO
We have chosen the datasets based on the following criteria:
- datasets where we do not need to do any special preprocessing so that it is easy to do in only one pipeline
- rather small datasets to ensure we do not need high computational power

In [5]:
# Dataset that seems useful
dataset_id = {
    "iris": 53, 
    "heart_disease": 45, 
    "wine_quality": 186, 
    "breast_cancer_wisconsin_diagnostic": 17, 
    "car_evaluation": 19, 
    # "abalone": 1, 
    "mushroom": 73, 
    "statlog" : 144, 
    "credit_approval" : 27, 
    "zoo" : 111, 
    "balance_scale" : 12, 
    "ilpd" : 225, 
    "acute_inflamations" : 184, 
    "ecoli" : 39, 
    "mammographic_mass" : 161, 
    "hayes_roth" : 44, 
    "habermans_survival" : 43, 
    "congress_voting_records" : 105, 
    "balloons" : 13, 
    "lenses" : 58, 
    "fertility" : 244, 
}

data_set_sorted = {}
for i, name in enumerate(sorted(dataset_id.keys())):
    data_set_sorted[name] = (dataset_id[name], i+1)

print(data_set_sorted)

{'abalone': (1, 1), 'acute_inflamations': (184, 2), 'balance_scale': (12, 3), 'balloons': (13, 4), 'breast_cancer_wisconsin_diagnostic': (17, 5), 'car_evaluation': (19, 6), 'congress_voting_records': (105, 7), 'credit_approval': (27, 8), 'ecoli': (39, 9), 'fertility': (244, 10), 'habermans_survival': (43, 11), 'hayes_roth': (44, 12), 'heart_disease': (45, 13), 'ilpd': (225, 14), 'iris': (53, 15), 'lenses': (58, 16), 'mammographic_mass': (161, 17), 'mushroom': (73, 18), 'statlog': (144, 19), 'wine_quality': (186, 20), 'zoo': (111, 21)}


# Import and preprocess datasets
For the preprocessing we will do the following steps:
1. Remove any missing values. In the article the following is written: "Given that our classifiers are not oriented to data with missing features, the missing inputs are treated as zero, which should not bias the comparison results." We therefore also decided to just remove missing values and to more focus on the full pipeline instead of single datasets. Another way could have been interpolation.
2. Encode categorical data into numerical data. This we have to do to use the classifiers later on.
3. Remove certain columns if they are highly correlated to others.
4. Split the data into a train and a test set. We will use a 80/20 split.
5. Scale the data so that we have zero mean and standard deviation of one. This is done with the Standard scaler.

In [6]:
# ordinal_encoder = OrdinalEncoder()
# X, y = import_dataset(dataset_id["iris"], ordinal_encoder)
# y = encode_categorical_features(y, ordinal_encoder)
# # split the dataset
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_SEED)

# Testing different classifiers

## K-Nearest Neighbor classifier

In [7]:
# https://towardsdatascience.com/gridsearchcv-for-beginners-db48a90114ee
knn_params = [{'knn__n_neighbors': [3, 5, 7, 9],
        'knn__weights': ['uniform', 'distance'],
        'knn__leaf_size': [15, 20]}]

## Support vector machine classifier

In [8]:
# params taken from here: https://www.geeksforgeeks.org/svm-hyperparameter-tuning-using-gridsearchcv-ml/
svm_params = [{'svm__C': [0.1, 1, 10, 100, 1000],  
        'svm__gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
        'svm__kernel': ['rbf']} ]

## Logistic regression classifier

In [9]:
# params taken from here: https://www.geeksforgeeks.org/how-to-optimize-logistic-regression-performance/
log_reg_params = [
    {'log_reg__penalty':['l1','l2','elasticnet','none'],
    #'log_reg__C' : np.logspace(-4,4,10),
    'log_reg__solver': ['lbfgs','newton-cg','liblinear','saga'],
    #'log_reg__max_iter'  : [100,2500,5000]
}
]

## K-means unsupervised classifier

In [10]:
kmeans_params = []

## Affinity propagation unsupervised classifier

In [11]:
affinity_propagation_params = [{"damping": [0.5, 0.7]}, 
          {"convergence_iter": [10, 15]},
          #{"preference": [0.01]}
          ]

# Train test loop

In [12]:
for name, id in data_set_sorted.items():
        print("\n" + "*"*100)
        print(f"Current dataset: {name}")
        ordinal_encoder = OrdinalEncoder()

        # Set up dataset
        X, y = import_dataset(dataset_id[name], ordinal_encoder)
        labels = np.unique(y)
        y = encode_categorical_features(y, ordinal_encoder)
        print(f"Dataset size: {len(X)}")
        print(f"Labels in dataset: {labels}")

        # split the dataset
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_SEED, stratify = y)
        print("*"*100, end="\n\n")

        # KNN
        clf_name = "knn"
        print("_"*100)
        print(f"Classifier: {clf_name}")
        # https://towardsdatascience.com/gridsearchcv-for-beginners-db48a90114ee
        knn = KNeighborsClassifier()
        train_acc, train_f1, test_acc, test_f1, cm_train, cm_test = test_classifier(knn, clf_name, knn_params, X_train, y_train, X_test, y_test)

        write(writer, clf_name, cm_train, "train", id[1])
        write(writer, clf_name, cm_test, "test", id[1])

        train_acc_dict, train_f1_dict, test_acc_dict, test_f1_dict = save_metrics_to_dict(clf_name, 
                                                                                          train_acc, 
                                                                                          train_f1, 
                                                                                          test_acc, 
                                                                                          test_f1, 
                                                                                          train_acc_dict, 
                                                                                          train_f1_dict, 
                                                                                          test_acc_dict, 
                                                                                          test_f1_dict)
        
        # SVM 
        svm = SVC()
        clf_name = "svm"
        print("_"*100)
        print(f"Classifier: {clf_name}")
        train_acc, train_f1, test_acc, test_f1, cm_train, cm_test = test_classifier(svm, clf_name, svm_params, X_train, y_train, X_test, y_test)
        write(writer, clf_name, cm_train, "train", id[1])
        write(writer, clf_name, cm_test, "test", id[1] )
        train_acc_dict, train_f1_dict, test_acc_dict, test_f1_dict = save_metrics_to_dict(clf_name, 
                                                                                          train_acc, 
                                                                                          train_f1, 
                                                                                          test_acc, 
                                                                                          test_f1, 
                                                                                          train_acc_dict, 
                                                                                          train_f1_dict, 
                                                                                          test_acc_dict, 
                                                                                          test_f1_dict)

        # Logistic regression
        log_reg = LogisticRegression()
        clf_name = "log_reg"
        print("_"*100)
        print(f"Classifier: {clf_name}")
        train_acc, train_f1, test_acc, test_f1, cm_train, cm_test = test_classifier(log_reg, 
                                                                                    clf_name, 
                                                                                    log_reg_params, X_train, y_train, X_test, y_test)
        write(writer, clf_name, cm_train, "train", id[1])
        write(writer, clf_name, cm_test, "test", id[1] )
        train_acc_dict, train_f1_dict, test_acc_dict, test_f1_dict = save_metrics_to_dict(clf_name, 
                                                                                          train_acc, 
                                                                                          train_f1, 
                                                                                          test_acc, 
                                                                                          test_f1, 
                                                                                          train_acc_dict, 
                                                                                          train_f1_dict, 
                                                                                          test_acc_dict, 
                                                                                          test_f1_dict)
       
        # K-means
        clf_name = "kmeans"
        print("_"*100)
        print(f"Classifier: {clf_name}")
        kmeans_params = [{"algorithm": ["lloyd", "elkan"]}]
        n_clusters = len(labels)
        print("k-means n cluster", n_clusters)
        kmeans_params.append({"n_clusters": [n_clusters]})
        train_acc, train_f1, test_acc, test_f1, cm_train, cm_test = clustering_classification(KMeans, clf_name, kmeans_params, X_train, y_train, X_test, y_test, RANDOM_SEED, K_FOLDS)
        write(writer, clf_name, cm_train, "train", id[1])
        write(writer, clf_name, cm_test, "test", id[1] )
        train_acc_dict, train_f1_dict, test_acc_dict, test_f1_dict = save_metrics_to_dict(clf_name, 
                                                                                          train_acc, 
                                                                                          train_f1, 
                                                                                          test_acc, 
                                                                                          test_f1, 
                                                                                          train_acc_dict, 
                                                                                          train_f1_dict, 
                                                                                          test_acc_dict, 
                                                                                          test_f1_dict)

        # Agglomerative clustering

        

        # Affinity propagation
        clf_name = "affinity_propagation"
        print("_"*100)
        print(f"Classifier: {clf_name}")
        train_acc, train_f1, test_acc, test_f1, cm_train, cm_test =  clustering_classification(AffinityPropagation, clf_name, affinity_propagation_params, X_train, y_train, X_test, y_test, RANDOM_SEED, K_FOLDS)
        write(writer, clf_name, cm_train, "train", id[1])
        write(writer, clf_name, cm_test, "test", id[1] )
        train_acc_dict, train_f1_dict, test_acc_dict, test_f1_dict = save_metrics_to_dict(clf_name, 
                                                                                          train_acc, 
                                                                                          train_f1, 
                                                                                          test_acc, 
                                                                                          test_f1, 
                                                                                          train_acc_dict, 
                                                                                          train_f1_dict, 
                                                                                          test_acc_dict, 
                                                                                          test_f1_dict)

        # Write metrics to tensorboard, step is dataset id
        writer.add_scalars("Train acc", train_acc_dict, id[1])
        writer.add_scalars("Test acc", test_acc_dict, id[1])
        writer.add_scalars("Train f1", train_f1_dict, id[1])
        writer.add_scalars("Test f1", test_f1_dict, id[1])


writer.close()


****************************************************************************************************
Current dataset: abalone
Dataset size: 4177
Labels in dataset: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 29]
****************************************************************************************************

____________________________________________________________________________________________________
Classifier: knn
Cross validation best parameters: {'knn__leaf_size': 15, 'knn__n_neighbors': 7, 'knn__weights': 'uniform'}
knn, Train accuracy = 0.40077821011673154, Test accuracy = 0.24043062200956938
knn, Train f1-score = 0.38253583148275055, Test f1-score = 0.22477344651495762
____________________________________________________________________________________________________
Classifier: svm
Cross validation best parameters: {'svm__C': 100, 'svm__gamma': 0.01, 'svm__kernel': 'rbf'}
svm, Train accuracy = 0.29242741694103563, Test accurac

KeyboardInterrupt: 

# SVM

In [ ]:
# svm = SVC()
# # params taken from here: https://www.geeksforgeeks.org/svm-hyperparameter-tuning-using-gridsearchcv-ml/
# params = [{'svm__C': [0.1, 1, 10, 100, 1000],  
#               'svm__gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
#               'svm__kernel': ['rbf']} ]

# test_classifier(svm, "svm", params, X_train, y_train, X_test, y_test)

# Linear Regression

In [ ]:
# log_reg = LogisticRegression()
# # params taken from here: https://www.geeksforgeeks.org/how-to-optimize-logistic-regression-performance/
# params = [
#     {'log_reg__penalty':['l1','l2','elasticnet','none'],
#     #'log_reg__C' : np.logspace(-4,4,10),
#     'log_reg__solver': ['lbfgs','newton-cg','liblinear','saga'],
#     'log_reg__max_iter'  : [100,2500,5000]
# }
# ]

# test_classifier(log_reg, "log_reg", params, X_train, y_train, X_test, y_test)

# K-means clustering classification

In [ ]:
# n_clusters = len(np.unique(y_train))
# params = [{"n_clusters": [n_clusters, int(n_clusters/2)]}]
# clf_name = "kmeans"

# train_acc, train_f1, test_acc, test_f1 = clustering_classification(KMeans, clf_name, params, X_train, y_train, X_test, y_test, RANDOM_SEED, K_FOLDS)
# train_acc_dict[clf_name] = train_acc
# train_f1_dict[clf_name] = train_f1
# test_acc_dict[clf_name] = test_acc
# test_f1_dict[clf_name] = test_f1

In [ ]:
# params = [{"damping": [0.5, 0.99]}, 
#           {"convergence_iter": [10, 15]},
#           #{"preference": [0.01]}
#           ]
# clf_name = "affinity_propagation"

# clustering_classification(AffinityPropagation, clf_name, params, X_train, y_train, X_test, y_test, RANDOM_SEED, K_FOLDS)
# train_acc_dict[clf_name] = train_acc
# train_f1_dict[clf_name] = train_f1
# test_acc_dict[clf_name] = test_acc
# test_f1_dict[clf_name] = test_f1

# Save logs

In [ ]:
# print(train_acc_dict)
# print(test_acc_dict)
# print(train_f1_dict)
# print(test_f1_dict)

# writer.add_scalars("Train acc", train_acc_dict)
# writer.add_scalars("Test acc", test_acc_dict)
# writer.add_scalars("Train f1", train_f1_dict)
# writer.add_scalars("Test f1", test_f1_dict)
# writer.close()